In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymsm.plotting import stackplot
from pymsm.simulation import MultiStateSimulator, default_update_covariates_function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of pymsm.event_specific_fitter failed: Traceback (most recent call last):
  File "/home/hagair/anaconda3/envs/pymsm/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/hagair/anaconda3/envs/pymsm/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/hagair/anaconda3/envs/pymsm/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/hagair/anaconda3/envs/pymsm/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 790, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/hagair/repo/pymsm/src/pymsm/event_specific_fitter.py", line 2, in <module>
    from sksurv.tree import Su

# Rossi - single state example

In [12]:
# Load rossi dataset
from lifelines.datasets import load_rossi

rossi = load_rossi()
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(rossi, duration_col="week", event_col="arrest")
baseline_hazard = cph.baseline_hazard_["baseline hazard"]
coefs = cph.params_

from pymsm.datasets import load_rossi_competing_risk_data

rossi_competing_risk_data, covariate_names = load_rossi_competing_risk_data()

### Define the full model ###
competing_risks_models_list = [
    {
        "origin_state": 1,
        "target_states": [2],
        "model_defs": [{"coefs": coefs, "baseline_hazard": baseline_hazard}],
    }
]

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[2],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_names,
)

# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=rossi_competing_risk_data.loc[0, covariate_names],
    origin_state=1,
    current_time=0,
    n_random_samples=1000,
    max_transitions=10,
    print_paths=False,
)


100%|██████████| 1000/1000 [00:00<00:00, 2253.43it/s]


# 2 state example - AIDS-SI

In [13]:
from pymsm.datasets import load_aidssi

data = load_aidssi()
data.head()


# TODO 


,patnr,time,status,cause,ccr5
1,1,9.106,1,AIDS,WW
2,2,11.039,0,event-free,WM
3,3,2.234,1,AIDS,WW
4,4,9.878,2,SI,WM
5,5,3.819,1,AIDS,WW


# Many-state example - EBMT

In [14]:
# Fit model from data and extract model parts for simulation

from pymsm.datasets import prep_ebmt_long
competing_risk_dataset, covariate_cols, states_labels = prep_ebmt_long()

from pymsm.multi_state_competing_risks_model import MultiStateModel
multi_state_model = MultiStateModel(
    dataset=competing_risk_dataset,
    terminal_states = [5, 6],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_cols,
    competing_risk_data_format=True,
)

multi_state_model.fit()


/home/hagair/anaconda3/envs/pymsm/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Fitting Model at State: 1
>>> Fitting Transition to State: 2, n events: 785
>>> Fitting Transition to State: 3, n events: 907
>>> Fitting Transition to State: 5, n events: 95
>>> Fitting Transition to State: 6, n events: 160
Fitting Model at State: 2
>>> Fitting Transition to State: 5, n events: 112
>>> Fitting Transition to State: 6, n events: 39
>>> Fitting Transition to State: 4, n events: 227
Fitting Model at State: 3
>>> Fitting Transition to State: 4, n events: 433
>>> Fitting Transition to State: 6, n events: 197
>>> Fitting Transition to State: 5, n events: 56
Fitting Model at State: 4
>>> Fitting Transition to State: 5, n events: 107
>>> Fitting Transition to State: 6, n events: 137


In [15]:
from pymsm.simulation import extract_competing_risks_models_list_from_msm

competing_risks_models_list = extract_competing_risks_models_list_from_msm(
    multi_state_model, verbose=True
)

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[5, 6],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_cols,
)


1 2 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af0cccc40>
1 3 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3b12d2b2b0>
1 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af0981cd0>
1 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af15074f0>
2 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af0cc4eb0>
2 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3b3031f430>
2 4 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af25c5370>
3 4 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af14dff10>
3 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af14dc730>
3 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af3e63160>
4 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af3df2340>
4 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af3e080a0>


In [16]:
# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=competing_risk_dataset.loc[0, covariate_cols],
    origin_state=1,
    current_time=0,
    n_random_samples=10,
    max_transitions=10,
    print_paths=True,
)

100%|██████████| 10/10 [00:00<00:00, 119.61it/s]

[1, 2, 4, 6]
[19.000075513725566, 14.999938494675956, 140.0000829825837]
[1, 3, 4, 6]
[10.000023167170138, 229.99997683282987, 3382.000080839735]
[1, 3, 4, 6]
[20.00008129857387, 19.999956128505705, 677.9999625729204]
[1, 2, 4, 6]
[34.00008530094555, 0.9999667058565649, 2183.999947993198]
[1, 2, 4, 6]
[23.00009063543851, 4.999948038096118, 245.00001380210853]
[1, 3, 4, 5]
[13.000073789691669, 96.99993779721429, 65.99998841309404]
[1, 6]
[49.00004513870243]
[1, 2, 5]
[15.000039447627858, 2609.000053021734]
[1, 3, 4, 5]
[13.000089482735042, 74.99991051726496, 581.0]
[1, 3, 6]
[10.000059089294318, 3.999952727188445]
